# 1. Load Libraries

In [98]:
import pandas as pd
import numpy as np
import pytorch_forecasting as pf
from tabulate import tabulate
from tqdm import tqdm
from torch.utils.data import random_split, TensorDataset
import torch
import matplotlib.pyplot as plt
from torch.utils.data import (ConcatDataset, DataLoader, Dataset, Subset,
                              random_split)
from torchvision import datasets, transforms


In [99]:
path = r"C:\Users\User\Downloads\WindPowerForecastingData TASK.xlsx\WindPowerForecastingData TASK.xlsx"
data = pd.read_excel(path, engine='openpyxl')
print(tabulate(data.head(), headers='keys', tablefmt='psql'))

+----+---------------+-------------+---------+-----------+---------+-----------+
|    | TIMESTAMP     |   TARGETVAR |     U10 |       V10 |    U100 |      V100 |
|----+---------------+-------------+---------+-----------+---------+-----------|
|  0 | 20120101 1:00 |   0         | 2.1246  | -2.68197  | 2.86428 | -3.66608  |
|  1 | 20120101 2:00 |   0.0548791 | 2.52169 | -1.79696  | 3.34486 | -2.46476  |
|  2 | 20120101 3:00 |   0.110234  | 2.67221 | -0.822516 | 3.50845 | -1.21409  |
|  3 | 20120101 4:00 |   0.165116  | 2.4575  | -0.143642 | 3.21523 | -0.355546 |
|  4 | 20120101 5:00 |   0.15694   | 2.2459  |  0.389576 | 2.95768 |  0.332701 |
+----+---------------+-------------+---------+-----------+---------+-----------+


In [100]:
data['TIMESTAMP'] = pd.to_datetime(data['TIMESTAMP'])
data = data.set_index('TIMESTAMP')
print(tabulate(data.head(), headers='keys', tablefmt='psql'))

+---------------------+-------------+---------+-----------+---------+-----------+
| TIMESTAMP           |   TARGETVAR |     U10 |       V10 |    U100 |      V100 |
|---------------------+-------------+---------+-----------+---------+-----------|
| 2012-01-01 01:00:00 |   0         | 2.1246  | -2.68197  | 2.86428 | -3.66608  |
| 2012-01-01 02:00:00 |   0.0548791 | 2.52169 | -1.79696  | 3.34486 | -2.46476  |
| 2012-01-01 03:00:00 |   0.110234  | 2.67221 | -0.822516 | 3.50845 | -1.21409  |
| 2012-01-01 04:00:00 |   0.165116  | 2.4575  | -0.143642 | 3.21523 | -0.355546 |
| 2012-01-01 05:00:00 |   0.15694   | 2.2459  |  0.389576 | 2.95768 |  0.332701 |
+---------------------+-------------+---------+-----------+---------+-----------+


In [101]:
# Convert DataFrame to tensor (dropping NaNs for simplicity)
data_tensor = torch.tensor(data.dropna().values, dtype=torch.float32)

# Calculate sizes
total_size = len(data_tensor)
train_size = int(total_size * 0.6)
val_size = int(total_size * 0.2)
test_size = total_size - train_size - val_size

print(f"Train size: {train_size}")
print(f"Validation size: {val_size}")
print(f"Test size: {test_size}")

# Create TensorDataset
dataset = TensorDataset(data_tensor)

# Split dataset
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

Train size: 10059
Validation size: 3353
Test size: 3353


In [102]:
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

print(f"Train size: {len(train_loader.dataset)}")
print(f"Validation size: {len(val_loader.dataset)}")
print(f"Test size: {len(test_loader.dataset)}")

Train size: 10059
Validation size: 3353
Test size: 3353


In [103]:
data.describe()

,TARGETVAR,U10,V10,U100,V100
count,16765.000000,16800.000000,16800.000000,16800.000000,16800.000000
mean,0.303185,0.935519,-0.250466,1.568721,-0.554659
std,0.289702,2.556424,2.919984,4.266318,5.028343
min,0.000000,-7.494252,-9.993858,-10.910809,-15.294866
25%,0.063340,-1.062387,-2.337600,-1.746650,-4.613866
50%,0.206841,0.788789,-0.039825,1.487808,0.132922
75%,0.479184,2.585812,1.927597,4.682327,3.488219
max,1.000000,11.117029,9.508798,16.988418,14.314416
